In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout
import numpy as np
import pandas as pd
import shutil
import time
import torch
#import cv2 as cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
import seaborn as sns
sns.set_style('darkgrid')
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
from IPython.core.display import display, HTML

In [3]:
c=0
for filename in os.listdir("../archive/images"):
    if filename.endswith(".jpg"):
        c+=1    
print(c)


10015


In [4]:
groundTruth = pd.read_csv('../archive/GroundTruth.csv')

In [5]:
groundTruth

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
10010,ISIC_0034316,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10011,ISIC_0034317,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10012,ISIC_0034318,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10013,ISIC_0034319,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#Check to see if the categories are mutually exclusive
sumRows = groundTruth[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]].sum(axis=1) 
#Add up each row. If mutually exclusive sum of each row should be 1

print(sumRows.sum(axis=0))
print(len(sumRows))

10015.0
10015


In [7]:
sumColumns = pd.DataFrame(groundTruth[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]].sum(axis=0))
sumColumns = sumColumns.rename(columns={0: "Total"})
sumColumns['Percent Total'] = sumColumns['Total'] / len(groundTruth)

In [8]:
sumColumns
#Very skewed data. Will need to do stratified

,Total,Percent Total
MEL,1113.0,0.111133
NV,6705.0,0.669496
BCC,514.0,0.051323
AKIEC,327.0,0.032651
BKL,1099.0,0.109735
DF,115.0,0.011483
VASC,142.0,0.014179


In [9]:
#trainingDf, testingDf= train_test_split(groundTruth, test_size=0.30, random_state=42, stratify = groundTruth[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]])
#Stratify on each outcome

#modified
trainingDf, testingDf = train_test_split( groundTruth, test_size=0.30, random_state=42, stratify = groundTruth[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]])
#Stratify on each outcome





In [10]:
# Double check to see if stratification worked on the train set
trainSumColumns = pd.DataFrame(trainingDf[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]].sum(axis=0))
trainSumColumns = trainSumColumns.rename(columns={0: "Total"})
trainSumColumns['Percent Total'] = trainSumColumns['Total'] / len(trainingDf)
trainSumColumns

,Total,Percent Total
MEL,779.0,0.111127
NV,4693.0,0.669472
BCC,360.0,0.051355
AKIEC,229.0,0.032668
BKL,769.0,0.109700
DF,81.0,0.011555
VASC,99.0,0.014123


In [11]:
# Double check to see if stratification worked on the test set
testSumColumns = pd.DataFrame(testingDf[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]].sum(axis=0))
testSumColumns = testSumColumns.rename(columns={0: "Total"})
testSumColumns['Percent Total'] = testSumColumns['Total'] / len(testingDf)
testSumColumns

,Total,Percent Total
MEL,334.0,0.111148
NV,2012.0,0.669551
BCC,154.0,0.051248
AKIEC,98.0,0.032612
BKL,330.0,0.109817
DF,34.0,0.011314
VASC,43.0,0.014309


In [12]:
#Just a simple test data set for setting up the models
test = trainingDf.iloc[1:15,:]

In [13]:
# Make a list to store all of the processed images
x_data = []

#Process all the images in the training data set.
for i in trainingDf.iloc[:,0]:
    path="../archive/images/" + i + ".jpg"
    
    image=tf.keras.preprocessing.image.load_img(
        path, grayscale=False, color_mode="rgb", target_size=None, interpolation="nearest")
    input_arr = tf.keras.preprocessing.image.img_to_array(image)

    x_data.append(input_arr)



In [14]:
#Start Building model

shape = x_data[0].shape

In [15]:
# Simple dummy model
def buildModel():
    model = keras.models.Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), activation="relu", input_shape=shape))
    model.add(Conv2D(filters=32, kernel_size=(5,5), activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(rate=0.25))
    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(rate=0.5))
    model.add(Dense(7, activation="softmax"))
    return model
    


In [16]:
m = buildModel()
m.summary()

2022-04-12 20:25:16.871207: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 446, 596, 32)      2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 442, 592, 32)      25632     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 221, 296, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 221, 296, 32)      0         
                                                                 
 flatten (Flatten)           (None, 2093312)           0         
                                                                 
 dense (Dense)               (None, 256)               535888128 
                                                        

In [17]:
m.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
#Get y-Data ready for model
y_train_final = np.asarray(trainingDf.loc[:, test.columns != 'image'])
print(y_train_final.shape)

#Get x_Data ready for model
x_data_final = np.asarray(x_data)
print(x_data_final.shape)

#I think that this scales the data
x_data_final = x_data_final.astype('float32')
x_data_final = x_data_final / 255.



(7010, 7)
(7010, 450, 600, 3)


In [21]:
epochs = 10
history = m.fit(x_data_final, y_train_final, epochs=epochs, batch_size=64)



Epoch 1/10
  7/110 [>.............................] - ETA: 1:40:10 - loss: 11583.1895 - accuracy: 0.4330

KeyboardInterrupt: 

In [22]:
x_data_final[0]

array([[[176., 125., 144.],
        [175., 126., 145.],
        [177., 129., 145.],
        ...,
        [178., 152., 163.],
        [175., 152., 160.],
        [176., 153., 161.]],

       [[173., 122., 137.],
        [174., 126., 142.],
        [177., 129., 145.],
        ...,
        [175., 152., 162.],
        [177., 151., 162.],
        [178., 152., 165.]],

       [[170., 122., 138.],
        [174., 126., 142.],
        [176., 128., 144.],
        ...,
        [177., 151., 162.],
        [176., 150., 161.],
        [177., 149., 161.]],

       ...,

       [[178., 131., 139.],
        [177., 130., 138.],
        [177., 131., 134.],
        ...,
        [167., 128., 131.],
        [168., 127., 133.],
        [171., 128., 135.]],

       [[178., 135., 145.],
        [178., 132., 142.],
        [177., 130., 136.],
        ...,
        [169., 127., 129.],
        [168., 126., 130.],
        [169., 126., 135.]],

       [[177., 134., 143.],
        [178., 132., 142.],
        [178., 1